In [7]:
import pandas as pd
import numpy as np
from scipy.interpolate import Rbf

# Load the Excel file
file_path = 'InputData.xlsx'
data = pd.read_excel(file_path, sheet_name='Sheet1')

# Extract the years and values from the dataset
years = data['Year'].values
values = data['Individuals using the Internet (% of population)'].values

# Adjust the years to represent December snapshots
years_adjusted = years + (11/12)

# Generate monthly points from Dec 2010 to March 2024
months_per_year = 12
num_months = int((years_adjusted.max() - years_adjusted.min()) * 12 + 4)
monthly_years_adjusted = np.linspace(years_adjusted.min(), years_adjusted.max() + (3 / 12), num_months)

# Use Gaussian Radial Basis Function (RBF) for smooth interpolation
rbf_quadratic = Rbf(years_adjusted, values, function='quintic')


monthly_values_rbf_precise = []
for i in range(len(years_adjusted) - 1):
    dec_current_value = values[i]
    dec_next_value = values[i + 1]
    
    # Interpolate for each year range
    rbf_interpolated = rbf_quadratic(np.linspace(years_adjusted[i], years_adjusted[i+1], months_per_year))
    
    rbf_clipped = np.clip(rbf_interpolated, dec_current_value, dec_next_value)
    
    monthly_values_rbf_precise.extend(rbf_clipped)


monthly_values_rbf_precise.append(values[-1])


extended_values_rbf = rbf_quadratic(monthly_years_adjusted[-3:])
monthly_values_rbf_precise.extend(extended_values_rbf)

# Generate the corresponding dates from Dec 2010 to March 2024
start_date = pd.to_datetime('2010-12-01')
monthly_dates_precise = pd.date_range(start=start_date, periods=len(monthly_values_rbf_precise), freq='M')

# Create a DataFrame and round values to two decimal places
final_monthly_data_precise = pd.DataFrame({
    'Month_Year': monthly_dates_precise.strftime('%b-%Y'),
    'Estimated Individuals using the Internet (% of population)': np.round(monthly_values_rbf_precise, 2)
})

# Save the final result to a CSV file
output_csv_precise_path = 'IUI_Monthly_Data.csv'
final_monthly_data_precise.to_csv(output_csv_precise_path, index=False)


C:\Users\hshar\AppData\Local\Temp\ipykernel_39128\1954864912.py:46: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_dates_precise = pd.date_range(start=start_date, periods=len(monthly_values_rbf_precise), freq='M')
